In [6]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.naive_bayes import MultinomialNB

In [7]:
# carregando dataset, adicionando os dados em um dataframe fltrando as para trabalhanr somente com a classe 0 e 1
iris = load_iris()
iris_data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_data['class'] = iris.target
iris_data = iris_data[iris_data['class'].isin([0, 1])]

In [8]:
# Discretizando os atributos do Iris para usar MultinomialNB
for col in iris_data.columns[:-1]: # iterando sobre todas as colunas exceto a coluna class
    iris_data[col] = pd.cut(iris_data[col], bins=3, labels=False, include_lowest=True, duplicates='drop')
    # iris_data[col] são os dados de cada coluna
    # bins=3 divide os dados em 3 bins (categorias)
    # 

In [ ]:
X = iris_data[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
y = iris_data['class']
clf = MultinomialNB() # Usando MultinomialNB
clf.fit(X, y)

In [4]:
def calculate_pi_explanation_loglinear(instance, clf, features):
    """Calcula uma PI-explicação de menor tamanho para uma instância utilizando um algoritmo log-linear.

    Args:
        instance: A instância para a qual se deseja calcular a PI-explicação.
        clf: O classificador Naive Bayes treinado.
        features: Nomes dos atributos do conjunto de dados.

    Returns:
        Uma lista de pares valor-atributo que constituem a PI-explicação.
    """

    # Calcula a probabilidade da classe prevista
    predicted_class = clf.predict([instance])[0]
    log_prob_predicted_class = clf.predict_log_proba([instance])[0][predicted_class]

    # Calcula a probabilidade da outra classe
    log_prob_other_class = clf.predict_log_proba([instance])[0][1 - predicted_class]

    # Calcula a diferença de probabilidade logarítmica para cada atributo
    delta_features = [(feature,
                       clf.feature_log_prob_[predicted_class][i] - clf.feature_log_prob_[1 - predicted_class][i])
                      for i, feature in enumerate(features)]

    # Ordena os atributos pela diferença de probabilidade logarítmica (maior primeiro)
    delta_features.sort(key=lambda x: x[1], reverse=True)

    # Seleciona os atributos que contribuem para a classe prevista
    pi_explanation = []
    threshold = log_prob_predicted_class - log_prob_other_class
    current_threshold = 0
    for feature, delta in delta_features:
        current_threshold += delta
        if current_threshold > threshold:
            pi_explanation.append((feature, instance[i]))
            break

    return pi_explanation

# Exemplo de uso
instance = iris_data.iloc[0].to_list()[:-1]  # Seleciona a primeira instância
pi_explanation = calculate_pi_explanation_loglinear(instance, clf, iris.feature_names)
print(f"PI-explicação Log-Linear para a instância {instance}: {pi_explanation}") 

c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


AttributeError: 'GaussianNB' object has no attribute 'feature_log_prob_'

In [ ]:
def enumerate_pi_explanations(instance, clf, features):
    """Enumera todas as PI-explicações para uma instância utilizando um algoritmo de atraso polinomial.

    Args:
        instance: A instância para a qual se deseja calcular as PI-explicações.
        clf: O classificador Naive Bayes treinado.
        features: Nomes dos atributos do conjunto de dados.

    Returns:
        Uma lista de todas as PI-explicações possíveis.
    """

    predicted_class = clf.predict([instance])[0]
    log_prob_predicted_class = clf.predict_log_proba([instance])[0][predicted_class]
    log_prob_other_class = clf.predict_log_proba([instance])[0][1 - predicted_class]

    delta_features = [(feature,
                       clf.feature_log_prob_[predicted_class][i] - clf.feature_log_prob_[1 - predicted_class][i])
                      for i, feature in enumerate(features)]
    delta_features.sort(key=lambda x: x[1], reverse=True)

    pi_explanations = []
    def backtrack(idx, current_explanation, current_threshold):
        if current_threshold > log_prob_predicted_class - log_prob_other_class:
            pi_explanations.append(current_explanation)
            return

        if idx >= len(delta_features):
            return

        feature, delta = delta_features[idx]
        current_explanation.append((feature, instance[idx]))
        backtrack(idx + 1, current_explanation.copy(), current_threshold + delta)
        current_explanation.pop()
        backtrack(idx + 1, current_explanation.copy(), current_threshold)

    backtrack(0, [], 0)

    return pi_explanations

# Exemplo de uso
instance = iris_data.iloc[1].to_list()[:-1]  # Seleciona a segunda instância
pi_explanations = enumerate_pi_explanations(instance, clf, iris.feature_names)
print(f"PI-explicações com atraso polinomial para a instância {instance}: {pi_explanations}")